In [ ]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

# 1️⃣ Load CSV
df = pd.read_csv('youtube_.csv')

# 2️⃣ Build directed grap
G = nx.DiGraph()
for _, row in df.iterrows():
    node_id = row['comment_id']
    parent_id = row['parent_id']
    G.add_node(node_id, text=row['text'], author=row['author'],
               sentiment=row['sentiment_class'], emotion=row['emotion'],
               likes=row['likes'])
    if pd.notna(parent_id):
        G.add_edge(parent_id, node_id)

# 3️⃣ Emotion color map
emotion_colors = {
    'anger': 'red', 'disgust': 'brown', 'fear': 'purple',
    'joy': 'yellow', 'neutral': 'gray', 'sadness': 'blue', 'surprise': 'orange'
}

# 4️⃣ Layout
pos = nx.spring_layout(G, k=0.3, iterations=50)

# 5️⃣ Extract node properties
node_x = []
node_y = []
node_color = []
node_size = []
node_text = []

for node in G.nodes():
    x, y = pos[node]
    data = G.nodes[node]
    node_x.append(x)
    node_y.append(y)
    color = emotion_colors.get(data.get('emotion', 'neutral'), 'gray')
    node_color.append(color)
    size = 10 + (data.get('likes', 0) * 2)
    node_size.append(size)
    text = f"Author: {data.get('author', 'Unknown')}<br>Text: {data.get('text', '')}<br>Sentiment: {data.get('sentiment', 'Unknown')}<br>Emotion: {data.get('emotion', 'Unknown')}<br>Likes: {data.get('likes', 0)}"
    node_text.append(text)

# 6️⃣ Edge positions
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

# 7️⃣ Create Plotly figure
fig = go.Figure()

# Add edges
fig.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'))

# Add nodes
fig.add_trace(go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    marker=dict(
        color=node_color,
        size=node_size,
        line_width=2),
    text=node_text,
    hoverinfo='text'))

# Layout settings
fig.update_layout(
    title='Comment Reply Network with Sentiment/Emotion',
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20,l=5,r=5,t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

# 8️⃣ Show interactive plot
fig.show()


KeyboardInterrupt: 